In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import acquire
import prepare

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
rawdf = acquire.get_zillow_data()

In [4]:
rawdf.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,tax_amount
0,"-118,740133","34,251502",4.00,2.00,"1,323.00","6,825.00",NaN,06111,53.00,"453,000.00","5,235.32"
1,"-118,217048","33,798657",4.00,2.00,"1,339.00","6,107.00",NaN,06037,40.00,"298,371.00","3,714.58"
2,"-118,531253","34,039467",2.00,2.00,"1,231.00","11,837.00",NaN,06037,67.00,"161,897.00","2,031.39"
3,"-117,613897","33,663005",5.00,3.00,"3,008.00","6,760.00",1.00,06059,22.00,"472,384.00","7,234.74"
4,"-118,393309","34,227422",2.00,1.00,996.00,"7,518.00",NaN,06037,77.00,"58,693.00",751.66


In [5]:
df = prepare.prep_zillow(rawdf)

In [6]:
df.shape

(47281, 23)

In [11]:
df.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,...,bathrooms_size,county_Los Angeles,county_Orange,county_Ventura,bedrooms_size_small,bedrooms_size_medium,bedrooms_size_large,bathrooms_size_small,bathrooms_size_medium,bathrooms_size_large
0,"-118,740133","34,251502",4.0,2.0,1323.0,6825.0,0.0,06111,53,453000.0,...,small,0,0,1,0,1,0,1,0,0
1,"-118,217048","33,798657",4.0,2.0,1339.0,6107.0,0.0,06037,40,298371.0,...,small,1,0,0,0,1,0,1,0,0
2,"-118,531253","34,039467",2.0,2.0,1231.0,11837.0,0.0,06037,67,161897.0,...,small,1,0,0,1,0,0,1,0,0
3,"-117,613897","33,663005",5.0,3.0,3008.0,6760.0,1.0,06059,22,472384.0,...,medium,0,1,0,0,0,1,0,1,0
6,"-118,689128","34,284379",3.0,2.5,1666.0,3301.0,0.0,06111,20,362552.0,...,small,0,0,1,0,1,0,1,0,0


## Simple Model
$$ y = mx + b $$

$$y = f(x)$$

In [8]:
scaled_df = df.copy()

In [9]:
scaler = MinMaxScaler()

In [10]:
scaler.fit(scaled_df[['square_feet', 'lot_size', 'age']])

MinMaxScaler()

In [11]:
scaled_df[['square_feet', 'lot_size', 'age']] = scaler.transform(scaled_df[['square_feet', 'lot_size', 'age']])

In [12]:
scaled_df.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,...,bathrooms_size,county_Los Angeles,county_Orange,county_Ventura,bedrooms_size_small,bedrooms_size_medium,bedrooms_size_large,bathrooms_size_small,bathrooms_size_medium,bathrooms_size_large
0,"-118,740133","34,251502",4.00,2.00,0.13,0.05,0.00,06111,0.38,"453,000.00",...,small,0,0,1,0,1,0,1,0,0
1,"-118,217048","33,798657",4.00,2.00,0.13,0.04,0.00,06037,0.28,"298,371.00",...,small,1,0,0,0,1,0,1,0,0
2,"-118,531253","34,039467",2.00,2.00,0.12,0.09,0.00,06037,0.48,"161,897.00",...,small,1,0,0,1,0,0,1,0,0
3,"-117,613897","33,663005",5.00,3.00,0.40,0.05,1.00,06059,0.15,"472,384.00",...,medium,0,1,0,0,0,1,0,1,0
4,"-118,393309","34,227422",2.00,1.00,0.08,0.06,0.00,06037,0.55,"58,693.00",...,small,1,0,0,1,0,0,1,0,0


In [13]:
s_train, s_validate, s_test = prepare.split(scaled_df)

In [14]:
pd.set_option('display.max_columns', None)

In [17]:
cols = ['square_feet', 'lot_size', 'has_pool', 'age', 'county_Los Angeles', 'county_Orange', 'bedrooms', 'bathrooms']

X_train = s_train[cols]
y_train = s_train.assessed_value

X_validate = s_validate[cols]
y_validate = s_validate.assessed_value

X_test = s_test[cols]
y_test = s_test.assessed_value

In [21]:
train_predictions = pd.DataFrame({
    'actual': s_train.assessed_value
}) 
validate_predictions = pd.DataFrame({
    'actual': s_validate.assessed_value
}) 

## Multiple  Regression + RFE

In [19]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
k = 6

# 1. Transform our X
rfe = RFE(lm, n_features_to_select=5)
rfe.fit(X_train, y_train)
print('selected top 8 features:', X_train.columns[rfe.support_])
X_train_rfe = rfe.transform(X_train)

selected top 8 features: Index(['square_feet', 'lot_size', 'age', 'county_Los Angeles', 'bedrooms'], dtype='object')


### Looping through k

In [22]:
for k in range(2,8):
    lm = LinearRegression()
    # 1. Transform our X
    rfe = RFE(lm, n_features_to_select=k)
    rfe.fit(X_train, y_train)
    # 2. Use the transformed x in our model
    X_train_rfe = rfe.transform(X_train)
    X_validate_rfe = rfe.transform(X_validate)
    lm.fit(X_train_rfe, y_train)
    
    # Make predictions
    train_predictions['multiple_rfe_k=', k] = lm.predict(X_train_rfe)
    validate_predictions['multiple_rfe_k=', k] = lm.predict(X_validate_rfe)
    k+=1

In [23]:
train_predictions.head()

,actual,"(multiple_rfe_k=, 2)","(multiple_rfe_k=, 3)","(multiple_rfe_k=, 4)","(multiple_rfe_k=, 5)","(multiple_rfe_k=, 6)","(multiple_rfe_k=, 7)"
27797,"339,675.00","427,543.61","457,229.19","429,248.28","421,741.62","417,489.23","411,993.11"
18599,"913,000.00","562,177.81","592,701.08","682,423.31","680,160.04","669,224.02","658,384.19"
512,"543,000.00","170,947.34","161,485.15","223,216.51","230,087.86","244,580.55","244,224.00"
18002,"547,000.00","348,856.82","382,724.94","387,402.68","384,078.60","383,429.19","379,395.83"
44818,"397,288.00","330,636.74","365,378.25","366,886.52","367,149.54","373,503.19","404,321.63"


In [24]:
train_predictions['baseline'] = y_train.mean()

In [25]:
def calculate_rmse(y_predicted):
    return mean_squared_error(train_predictions.actual, y_predicted, squared = False)

train_predictions.apply(calculate_rmse).sort_values()

actual                       0.00
(multiple_rfe_k=, 7)   232,439.10
(multiple_rfe_k=, 6)   232,815.36
(multiple_rfe_k=, 5)   233,566.15
(multiple_rfe_k=, 4)   233,620.68
(multiple_rfe_k=, 3)   235,668.09
(multiple_rfe_k=, 2)   236,841.47
baseline               274,767.86
dtype: float64

In [27]:
validate_predictions['baseline'] = y_validate.mean()

In [28]:
validate_predictions.head()

,actual,"(multiple_rfe_k=, 2)","(multiple_rfe_k=, 3)","(multiple_rfe_k=, 4)","(multiple_rfe_k=, 5)","(multiple_rfe_k=, 6)","(multiple_rfe_k=, 7)",baseline
46740,"629,682.00","321,471.41","304,925.54","346,627.95","339,405.72","347,812.14","346,171.45","406,783.72"
18140,"187,026.00","469,130.60","498,559.38","478,239.42","481,696.22","481,770.43","472,452.04","406,783.72"
1508,"768,308.00","819,603.59","786,081.38","777,394.88","779,554.62","732,972.62","749,730.97","406,783.72"
7139,"188,466.00","403,396.11","384,394.31","394,463.66","386,548.95","414,469.99","409,149.96","406,783.72"
30114,"73,980.00","262,122.37","248,808.16","280,281.54","285,707.84","276,369.40","273,728.21","406,783.72"


In [29]:
def calculate_rmse(y_predicted):
    return mean_squared_error(validate_predictions.actual, y_predicted, squared = False)

validate_predictions.apply(calculate_rmse).sort_values()

actual                       0.00
(multiple_rfe_k=, 7)   231,828.39
(multiple_rfe_k=, 6)   232,008.57
(multiple_rfe_k=, 5)   232,843.79
(multiple_rfe_k=, 4)   233,020.30
(multiple_rfe_k=, 3)   234,724.57
(multiple_rfe_k=, 2)   236,085.70
baseline               273,944.34
dtype: float64

In [30]:
# k=7 is the best

231828-232439

-611

**K=7 has the lowest rmse, highest performance**

## Polynomial Features

In [31]:
train_pred = pd.DataFrame({
    'actual': s_train.assessed_value
}) 
validate_pred = pd.DataFrame({
    'actual': s_validate.assessed_value
}) 

In [48]:
from sklearn.preprocessing import PolynomialFeatures

# 1. Generate Polynomial Features, k=2
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [49]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 2'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 2'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction
27797,"339,675.00","407,658.98","533,287.61","537,645.58"
18599,"913,000.00","407,658.98","636,001.73","665,864.73"
512,"543,000.00","407,658.98","337,269.36","352,657.34"
18002,"547,000.00","407,658.98","410,062.86","410,121.81"
44818,"397,288.00","407,658.98","373,780.36","380,158.07"


In [35]:
validate_pred.head()

,actual,baseline,polynomial degree 2
46740,"629,682.00","406,783.72","287,334.63"
18140,"187,026.00","406,783.72","430,215.98"
1508,"768,308.00","406,783.72","696,422.90"
7139,"188,466.00","406,783.72","373,457.15"
30114,"73,980.00","406,783.72","334,905.86"


In [40]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 2'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 2'], squared = False)

In [38]:
train_rmse_b = mean_squared_error(train_pred.actual,train_pred['baseline'], squared = False)
validate_rmse_b = mean_squared_error(validate_pred.actual,validate_pred['baseline'], squared = False)

In [41]:
train_rmse, validate_rmse

(225563.1468062608, 224965.8539400748)

In [42]:
validate_rmse-train_rmse

-597.292866186006

In [39]:
train_rmse_b, validate_rmse_b

(274767.8629314713, 273944.3392279506)

In [50]:
# Interaction terms only

poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)
lm = LinearRegression()
lm.fit(X_train_poly, y_train)

X_validate_poly = poly.transform(X_validate)
train_pred['polynomial degree 2 only interaction'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 2 only interaction'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction
27797,"339,675.00","407,658.98","533,287.61","537,645.58"
18599,"913,000.00","407,658.98","636,001.73","665,864.73"
512,"543,000.00","407,658.98","337,269.36","352,657.34"
18002,"547,000.00","407,658.98","410,062.86","410,121.81"
44818,"397,288.00","407,658.98","373,780.36","380,158.07"


In [51]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 2 only interaction'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 2 only interaction'], squared = False)
train_rmse, validate_rmse

(225829.11823267405, 225167.73661349496)

In [52]:
validate_rmse-train_rmse

-661.3816191790975

In [53]:
# k=3
poly = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [54]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 3'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 3'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction,polynomial degree 3
27797,"339,675.00","407,658.98","533,287.61","537,645.58","582,286.17"
18599,"913,000.00","407,658.98","636,001.73","665,864.73","638,633.23"
512,"543,000.00","407,658.98","337,269.36","352,657.34","271,184.30"
18002,"547,000.00","407,658.98","410,062.86","410,121.81","383,339.48"
44818,"397,288.00","407,658.98","373,780.36","380,158.07","362,223.42"


In [55]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 3'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 3'], squared = False)
train_rmse, validate_rmse

(222833.2843929147, 223820.32945259503)

In [56]:
# k=4
poly = PolynomialFeatures(degree=4, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [57]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 4'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 4'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction,polynomial degree 3,polynomial degree 4
27797,"339,675.00","407,658.98","533,287.61","537,645.58","582,286.17","583,336.55"
18599,"913,000.00","407,658.98","636,001.73","665,864.73","638,633.23","719,444.55"
512,"543,000.00","407,658.98","337,269.36","352,657.34","271,184.30","189,598.25"
18002,"547,000.00","407,658.98","410,062.86","410,121.81","383,339.48","386,924.89"
44818,"397,288.00","407,658.98","373,780.36","380,158.07","362,223.42","391,558.28"


In [58]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 4'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 4'], squared = False)
train_rmse, validate_rmse

(220599.9432621597, 228423.90903466812)

**K=3 has the lowest rmse**

## Lasso-Lars

In [59]:
from sklearn.linear_model import LassoLars
# create the model object
lars = LassoLars(alpha=0)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)

pd.Series(lars.coef_, index=X_train.columns).sort_values()

lot_size              -156,273.52
bedrooms               -52,310.25
county_Los Angeles     -50,624.11
county_Ventura               0.00
county_Orange           12,648.51
has_pool                34,981.79
bathrooms               40,450.36
age                     71,769.57
square_feet          1,166,547.13
dtype: float64

In [60]:
# create the model object
lars = LassoLars(alpha=1)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)
X_validate_pred_lars = lars.predict(X_validate)
# Add lassolars predictions to our predictions DataFrame
train_pred['lasso_lars'] = X_train_pred_lars
validate_pred['lasso_lars'] = X_validate_pred_lars

In [61]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['lasso_lars'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['lasso_lars'], squared = False)
train_rmse, validate_rmse

(232417.49582058165, 231827.48777021127)

In [62]:
validate_rmse-train_rmse

-590.0080503703794

## Generalized Linear Model

In [63]:
from sklearn.linear_model import TweedieRegressor

# create the model object
glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data
glm.fit(X_train, y_train)

# predict train
X_train_predict_glm = glm.predict(X_train)
X_validate_predict_glm = glm.predict(X_validate)
# Add lassolars predictions to our predictions DataFrame
train_pred['glm'] = X_train_predict_glm
validate_pred['glm'] = X_validate_predict_glm

In [64]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['glm'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['glm'], squared = False)
train_rmse, validate_rmse

(235324.13823328284, 234667.38804766495)

In [65]:
validate_rmse-train_rmse

-656.750185617886

## Evaluation